In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import inspect

In [12]:
n_qubits = 16
dev = qml.device("qiskit.aer", wires=n_qubits)

DeviceError: Device qiskit.aer does not exist. Make sure the required plugin is installed.

In [13]:
@qml.qnode(dev, interface="torch")
def qnode(inputs, conv1, entangle1, pool1, conv2, entangle2):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    for i in range(n_qubits):
        qml.Rot(*conv1[i], wires=i)
    for i in range(n_qubits):
        qml.CRX(entangle1[i][0], wires=[i, (i + 1) % n_qubits])
    kept = list(range(0, n_qubits, 2))
    for i, w in enumerate(kept):
        qml.Rot(*pool1[i], wires=w)
    for i, w in enumerate(kept):
        qml.Rot(*conv2[i], wires=w)
    for i in range(len(kept)):
        qml.CRX(entangle2[i][0], wires=[kept[i], kept[(i + 1) % len(kept)]])
    return [qml.expval(qml.PauliZ(q)) for q in kept[:4]]

In [14]:
weight_shapes = {
    "conv1": (16, 3),
    "entangle1": (16, 1),
    "pool1": (8, 3),
    "conv2": (8, 3),
    "entangle2": (8, 1),
}

In [15]:
dummy_input = np.zeros(16, dtype=np.float32)
dummy_weights = []
for name in inspect.signature(qnode).parameters:
    if name == "inputs":
        continue
    shape = weight_shapes[name]
    dummy_weights.append(np.zeros(shape, dtype=np.float32))

In [16]:
# Call the circuit to populate dev._circuit
qnode(dummy_input, *dummy_weights)

# Now draw it with Qiskit's drawer
circuit_diagram = dev._circuit.draw(output="mpl")

# Save to file
circuit_diagram.savefig("qcnn_circuit_qiskit_style.pdf", bbox_inches="tight")
plt.show()


/Users/lopez/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pennylane/math/interface_utils.py:136: UserWarning: Contains tensors of types {'torch', 'autograd'}; dispatch will prioritize TensorFlow, PyTorch, and Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


AttributeError: DefaultQubit has no attribute '_circuit'. You may be looking for a property or method present in the legacy device interface. Please consult the DefaultQubit documentation for an updated list of public properties and methods.